In [1]:
import torch
import random
import numpy as np
import os

# 시드값 고정
seed = 50
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)

In [2]:
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.enabled = False

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
# 데이터 경로
data_path = './data/'

# 훈련, 검증, 테스트 경로 설정
train_path = data_path + 'train/'
valid_path = data_path + 'val/'
test_path = data_path + 'test/'

In [5]:
from torchvision import transforms

# 훈련 데이터용 변환기
transform_train = transforms.Compose([
    transforms.Resize((250, 250)), # 이미지 크기 조정
    transforms.CenterCrop(180), # 중앙 이미지 확대
    transforms.RandomHorizontalFlip(0.5), # 좌우 대칭
    transforms.RandomVerticalFlip(0.2), # 상하 대칭
    transforms.RandomRotation(20), # 이미지 회전
    transforms.ToTensor(), # 텐서 객체로 변환
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225)) # 정규화
])

# 테스트 데이터용 변환기
transform_test = transforms.Compose([
    transforms.Resize((250, 250)),
    transforms.CenterCrop(180),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))
])

In [8]:
from torchvision.datasets import ImageFolder

# 훈련 데이터셋
datasets_train = ImageFolder(root=train_path, transform=transform_train)
# 검증 데이터셋 
datasets_valid = ImageFolder(root=valid_path, transform=transform_test)

In [9]:
def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    
# 제네레이터 시드값 고정
g = torch.Generator()
g.manual_seed(0)

In [10]:
from torch.utils.data import DataLoader

batch_size = 8

loader_train = DataLoader(
    dataset=datasets_train, batch_size=batch_size,
    shuffle=True, worker_init_fn=seed_worker, 
    generator=g, num_workers=2
)
loader_valid = DataLoader(
    dataset=datasets_valid, batch_size=batch_size,
    shuffle=False, worker_init_fn=seed_worker,
    generator=g, num_workers=2
)

In [11]:
!pip install efficientnet-pytorch==0.7.1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from efficientnet_pytorch import EfficientNet

class Baseline_EfficientNet(pl.LightningModule):
    def __init__(self, num_classes):
        super().__init__()
        self.model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2).to(device) 
        
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.forward(x)
        loss = nn.CrossEntropyLoss()(y_hat, y)
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss, loss_dict = self.common_step(batch, batch_idx)     
        self.log("validation/loss", loss)
        for k, v in loss_dict.items():
            self.log("validation_" + k, v.item())
            
        return loss
    
    def configure_optimizers(self):    
        return torch.optim.Adam(self.model.parameters(), lr=0.01)

    def train_dataloader(self):
        return loader_train

    def val_dataloader(self):
        return loader_valid
